# Numerical solution of the 2d Poisson equation 

Numerical solution of the 2d Poisson equation on the rectangle $[0,1]\times [0,1]$:

\begin{align*}
  \Delta u  &= f, \qquad x\in[0,1],y\in[0,1]  \\
  u|_{\partial \Omega}&=g(x,y)
\end{align*}


Finite difference approximation (5 point formula):

\begin{align*}
    U_{i+1,j}+U_{i-1,j}+U_{i,j+1}+U_{i,j-1}-4U_{i,j}=h^2f_{i,j}
\end{align*}

* Import the necessary libraries for computations and plotting

In [ ]:
%matplotlib inline
import numpy as np
import scipy.sparse as sparse             # Sparse matrices
from scipy.sparse.linalg import spsolve   # Linear solver for sparse matrices
import matplotlib.pyplot as plt
import pandas as pd  
newparams = {'figure.figsize': (10.0, 5.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 12}
plt.rcParams.update(newparams)

In [ ]:
def plot_surface(x,y,U):
    '''
    Plot surfaces 
    '''
    plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(x, y, U)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [ ]:
def laplacian(M):
    '''
    Construct the Discrete Laplacian
    '''
    Mi = M-1       # Number of inner points in each direction
    Mi2 = Mi**2    # Number of inner points in total

    # Construct a sparse A-matrix
    B = sparse.diags([1,-4,1],[-1,0,1],shape=(Mi, Mi), format="lil")
    A = sparse.kron(sparse.eye(Mi), B)
    C = sparse.diags([1,1],[-Mi,Mi],shape=(Mi2, Mi2), format="lil")
    A = (A+C).tocsr() # Konverter til csr-format (necessary for spsolve) 
    return A

In [ ]:
A = laplacian(4)
print(A.toarray())

Construct a class for the problem, the solver included

In [ ]:
class Poisson(object):
    '''
    Class for the Poisson problem Delta u = f on Omega, with u=g at the boundaries. 
    '''
    def __init__(self, f, g=0):
        self.f = f   # Right hand side
        self.g = g   # Boundary values

    def solve(self, M=10):
        '''
        Solve Poisson problem with the 5-point formula
        M: Number of grid intervals in each direction
        '''

        # Make the grid
        x = np.linspace(0, 1, M+1)
        y = np.linspace(0, 1, M+1) 
        h = 1/M

        # Inner points
        xi = x[1:-1]       
        yi = y[1:-1] 
        Xi, Yi = np.meshgrid(xi, yi)
        Mi = M-1          # Number of inner points in each direction

        A = laplacian(M)
        
        # Construct the right hand side directly on the grid 
        # To be consistent with the natural choice of axes, U[j,i] \approx u(x_i,y_j). 
        
        b = np.zeros((Mi,Mi))
        b[0,0:Mi]    -= self.g(xi,0.)       # lower boundary
        b[Mi-1,0:Mi] -= self.g(xi,1.)       # upper boundary
        b[0:Mi,0]    -= self.g(0.,yi)       # left  boundary
        b[0:Mi,Mi-1] -= self.g(1.,yi)       # right boundary
        b += h**2*self.f(Xi,Yi)         
        b = b.flatten() 
        

        # Solve the linear system (sparse solver)
        Ui = spsolve(A,b)

        # Include the boundary values
        U = np.zeros((M+1,M+1))
        U[0,0:M+1] = self.g(x,0.)
        U[M,0:M+1] = self.g(x,1.)
        U[0:M+1,0] = self.g(0.,y)
        U[0:M+1,M] = self.g(1.,y)
        U[1:-1,1:-1] = Ui.reshape(M-1,M-1)

        X, Y = np.meshgrid(x,y)
        return X, Y, U

## Test the code. 

Try the code on a problem for which the code will produce the exact answer, e.g. when $u$ is a polynonomial of degree 3 or less. 

In [ ]:
f = lambda x,y : 0
g = lambda x,y : x
pde = Poisson(f, g)
x, y, U = pde.solve(4);
plot_surface(x, y, U)
print(f'Error: {np.max(np.abs(g(x,y)-U))}') 
print(U)
print(g(x,y))

## Test of accuracy
* Make a test example.
* Here we force the exact solution to be $u=x^2\cos(5y)$
* We compute the corresponding $f$: 
\begin{align}
 f(x)&=\partial_x^2 u + \partial_y^2 u\\
 &=2\cos(5y)-25x^2\cos(5y)
\end{align}

In [ ]:
# Exact solution
def u(x,y):
    return x**2*np.cos(5*y)

# Forcing function
def f(x,y):
    return 2*np.cos(5*y)-25*x**2*np.cos(5*y)

# Boundary conditions
def g(x,y):
    return u(x,y)

ex1 = Poisson(f, g)  

* Solve it. 
* Plot the solution.

In [ ]:
M = 100
# Solution of the problem
x, y, U = ex1.solve(M)

# Plot of the solution
plot_surface(x, y, U)

* Plot the error

In [ ]:
err = g(x,y) - U
# plot of the error
plot_surface(x, y, np.abs(err))
print(f'Max error: {np.max(np.abs(err)):.2e}')

* Verify the convergence result by successively refining the grid. 
* We measure convergence in the max / $L^\infty$-norm

In [ ]:
# Function operating a convergence study
def convergence(pde, u_exact): 
    P = 4
    Hconv = np.zeros(P)
    Emax = np.zeros(P)
    M = 5
    for p in range(P):
        x, y, U = pde.solve(M)
        Emax[p] = np.max(np.abs(u_exact(x,y)-U))
        Hconv[p] = 1./M
        M = 2*M
    orderMax = np.polyfit(np.log(Hconv),np.log(Emax),1)[0]
    return Hconv, Emax, orderMax

In [ ]:
# Plot of the convergence plot
H, EM, pM = convergence(ex1, u)
Rate=np.zeros(np.size(EM))
Rate[1:]=np.log10(EM[1:]/EM[:-1])/np.log10(H[1:]/H[:-1])
pd.options.display.float_format = '{:.5f}'.format
df = pd.DataFrame(data={'h': H, 'Error': EM ,'Rate':Rate}) 
df


In [ ]:
plt.figure()
plt.loglog(H,EM,'o-', label='p={:.2f} in max norm'.format(pM))
plt.grid('on')
plt.xlabel('h')
plt.ylabel('error')
plt.legend();

## The advantage of using sparse matrices

Demonstrate this with the following example: 

Choose $M=100$ (less if you have a slow computer and not much memory).

Run the code below. A full matrix is constructed from the sparse. 
The linear system is then solved by a full and a sparse solver, and the memory and cpu-time used measured for both. 

In [ ]:
import time,sys
# Fullt system

M = 50
A = laplacian(M)            # A sparse matrix
b = np.random.rand((M-1)**2)

Af = A.toarray()            # Convert to a full matrix
print('Memory use of the full   matrix: {:d}'.format(Af.data.nbytes))
print('Memory use of the sparse matrix: {:d}'.format(A.data.nbytes))
print('Factor: {:.2e}'.format(Af.data.nbytes/A.data.nbytes))

In [ ]:
start = time.time()
U = np.linalg.solve(Af, b)            # Solve the system with a full solver
ferdig = time.time()
time_full = ferdig - start
print('Time used for a full solver:', time_full)

In [ ]:
# sparse system
start = time.time()
U = spsolve(A, b)            # Solve the system with a sparse solver
ferdig = time.time()
time_sparse = ferdig - start
print('Time used for a sparse solver:', time_sparse)
print('Factor: {:.2e}'.format(time_full/time_sparse))